### Dependencies

In [2]:
#importo librerias
import gzip
import pandas as pd
import urllib
import tarfile
import urllib.request
import numpy as np
import random
import json
import bisect


### Data Sets

In [3]:
url_item_data = "https://meli-data-challenge.s3.amazonaws.com/2020/item_data.jl.gz"
url_train_data = "https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz"

In [4]:
train_data = []
with urllib.request.urlopen(url_train_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    train_data.append(json.loads(line.strip().decode('utf-8')))

In [5]:
df = pd.DataFrame(train_data)
df.head()

,user_history,item_bought
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207


In [6]:
item_data = []
with urllib.request.urlopen(url_item_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    item_data.append(json.loads(line.strip().decode('utf-8')))

itemdf = pd.DataFrame(item_data)

In [7]:
itemdf.head()

,item_id,title,domain_id,product_id,price,category_id,condition
0,111260,Casa Sola En Venta Con Gran Patio Solo Pago De...,MLM-INDIVIDUAL_HOUSES_FOR_SALE,None,1150000.00,MLM170527,new
1,871377,Resident Evil Origins Collection Nintendo Swit...,MLM-VIDEO_GAMES,15270800,1392.83,MLM151595,new
2,490232,Falda De Imitación Piel Negra,MLM-SKIRTS,None,350.00,MLM7697,new
3,1150706,Powercolor Red Devil Radeon Rx 580 8gb Gddr5,MLM-GRAPHICS_CARDS,None,3200.00,MLM9761,used
4,934912,Laptop Hp Nx6320 Core Duo Con Puerto Db9 Windo...,MLM-NOTEBOOKS,None,1599.00,MLM1652,used


### Exploración de los Data Sets

In [8]:
print('Compras totales en el dataset: ', len(df))
print('-'*50)
print('Item comprados (item_bought) únicos: ', len(df.item_bought.unique()))
print('-'*50)
print('Proporción de items únicos en el dataset: ', ( len(df.item_bought.unique())/len(df))*100)

Compras totales en el dataset:  413163
--------------------------------------------------
Item comprados (item_bought) únicos:  64928
--------------------------------------------------
Proporción de items únicos en el dataset:  15.714863141181567


### Unión de los data sets por item bought

In [9]:
df_joined = pd.merge(df, itemdf, left_on='item_bought', right_on='item_id', how='left')
df_joined.shape

(413163, 9)

### Exploración y curación del data set

#### **Manejo de datos faltantes**

In [10]:
# Check for nulls
df_joined.isna().sum()

user_history         0
item_bought          0
item_id              0
title                0
domain_id            0
product_id      305666
price                0
category_id          0
condition            0
dtype: int64

In [11]:
#Solo la columna 'product_id' presenta valores nulos. Y un considerable numero: 305666 de 413163. Por ello, la eliminamos:

del df_joined['product_id']

#### **Eliminar registros duplicados**

Por el momento no se observan registros duplicados. 

### División del data set según Mexico o Brasil. 

In [12]:
df_joined[['country', 'domain']] = df_joined['domain_id'].str.split('-', expand=True)
df_joined.head()

,user_history,item_bought,item_id,title,domain_id,price,category_id,condition,country,domain
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,new,MLB,SMARTWATCHES
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737,228737,Bomba Eletrica Tira Leite Materno Bivolt G-tech,MLB-MILK_EXTRACTORS,169.00,MLB264021,new,MLB,MILK_EXTRACTORS
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110,1909110,"Kit Youtuber Tripé 1,20 Microfone Lapela Anel ...",MLB-CELLPHONE_ACCESSORIES,300.00,MLB5092,new,MLB,CELLPHONE_ACCESSORIES
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370,1197370,Leia A Descrição Por Favor - Maquininha Point ...,MLB-CARD_PAYMENT_TERMINALS,16.90,MLB277951,new,MLB,CARD_PAYMENT_TERMINALS
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207,2049207,Smartwatch Xiaomi Amazfit Bip Huami A1608 Orig...,MLB-SMARTWATCHES,355.99,MLB135384,new,MLB,SMARTWATCHES


In [13]:
print('Publicaciones totales: ', len(itemdf))
print('-'*50)
print('Total de dominios: ', len(df_joined.domain.unique()))
print('-'*50)
print('País en el que se realizo la compra'+ '\n'*1, df_joined.country.value_counts())

Publicaciones totales:  2102277
--------------------------------------------------
Total de dominios:  2275
--------------------------------------------------
País en el que se realizo la compra
 MLB    354907
MLM     58256
Name: country, dtype: int64


In [14]:
df_mlb = df_joined[df_joined['country'] == 'MLB']
df_mlm = df_joined[df_joined['country'] == 'MLM']

print(df_mlb.shape)
print(df_mlm.shape)

df_mlb

(354907, 10)
(58256, 10)


,user_history,item_bought,item_id,title,domain_id,price,category_id,condition,country,domain
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,new,MLB,SMARTWATCHES
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737,228737,Bomba Eletrica Tira Leite Materno Bivolt G-tech,MLB-MILK_EXTRACTORS,169.00,MLB264021,new,MLB,MILK_EXTRACTORS
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110,1909110,"Kit Youtuber Tripé 1,20 Microfone Lapela Anel ...",MLB-CELLPHONE_ACCESSORIES,300.00,MLB5092,new,MLB,CELLPHONE_ACCESSORIES
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370,1197370,Leia A Descrição Por Favor - Maquininha Point ...,MLB-CARD_PAYMENT_TERMINALS,16.90,MLB277951,new,MLB,CARD_PAYMENT_TERMINALS
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207,2049207,Smartwatch Xiaomi Amazfit Bip Huami A1608 Orig...,MLB-SMARTWATCHES,355.99,MLB135384,new,MLB,SMARTWATCHES
...,...,...,...,...,...,...,...,...,...,...
413157,"[{'event_info': 912949, 'event_timestamp': '20...",459011,459011,Torneira Cozinha Gourmet Monocomando Mangueira...,MLB-KITCHEN_FAUCETS,399.90,MLB270023,new,MLB,KITCHEN_FAUCETS
413159,"[{'event_info': 289961, 'event_timestamp': '20...",1845503,1845503,Kit Unhas Gel Uv Acrigel Mini Lixa Eletrica Ca...,MLB-GEL_NAIL_KITS,169.57,MLB196796,new,MLB,GEL_NAIL_KITS
413160,"[{'event_info': 'ALUGUEL BOB CAT', 'event_time...",2022477,2022477,Bateria Celular Positivo Twist S430 S430b Bt-s...,MLB-CELLPHONE_BATTERIES,46.88,MLB3812,new,MLB,CELLPHONE_BATTERIES
413161,"[{'event_info': 'XAOMI', 'event_timestamp': '2...",1111021,1111021,Capa Anti Queda Xiaomi Redmi Mi 9t/ K20 + Pelí...,MLB-CELLPHONE_COVERS,22.99,MLB5095,new,MLB,CELLPHONE_COVERS


### División del set de BRASIL en train, validation y test


*   Usamos el de Brasil xq es el más grande de ambos dataset y además, como deseamos analizar texto, necesitamos que dicho texto esté todo en el mismo idioma (trabajaremos con las columnas en portugués). 



###### **Empleamos sklearn para la división**

In [15]:
# Separamos el target (variable a predecir) del conjunto de datos a utilizar para predecirla

xtrain = df_mlb.drop(['item_bought'], axis=1)
labels = df_mlb['item_bought']

In [16]:
from sklearn.model_selection import train_test_split

x, x_test, y, y_test = train_test_split(xtrain,labels,test_size=0.2,train_size=0.8)
x_train, x_validation, y_train, y_validation = train_test_split(x,y,test_size = 0.25,train_size =0.75)

In [17]:
print('Size set for train: ', x_train.shape)
print('Size set for validation: ', x_validation.shape)
print('Size set for test: ', x_test.shape)
print('-'*50)
print('Size labels for train: ', y_train.shape)
print('Size labels for validation: ', y_validation.shape)
print('Size labels for test: ', y_test.shape)

Size set for train:  (212943, 9)
Size set for validation:  (70982, 9)
Size set for test:  (70982, 9)
--------------------------------------------------
Size labels for train:  (212943,)
Size labels for validation:  (70982,)
Size labels for test:  (70982,)


### Análisis de los campos de texto. 


*   Realizaremos el análisis sobre el dataset de training (x_train,y_train).  




#### Primero desglosamos la columna user_history del df de training

In [18]:
#x_train=x_train.sample(1000)


In [19]:
##Agregamos columna "user_id". 
x_train['user_id']=x_train.index
x_train['user_id2']=x_train['user_id']

C:\Users\pipin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\pipin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
x_train.set_index(x_train.user_id,inplace=True)

x_train.head()

,user_history,item_id,title,domain_id,price,category_id,condition,country,domain,user_id,user_id2
user_id,,,,,,,,,,,
236116,"[{'event_info': 'FIAT STRADA 2015 DUPLA', 'eve...",802603,Câmera Ip Robô Visão Noturna Hd Wifi Sem Fio G...,MLB-SURVEILLANCE_CAMERAS,169.99,MLB7073,new,MLB,SURVEILLANCE_CAMERAS,236116,236116
301466,"[{'event_info': 'COROLLA 2012 COFAP', 'event_t...",1488969,Militec-1 Condicionador De Metais 200ml-100%or...,MLB-VEHICLE_PARTS,69.90,MLB194028,new,MLB,VEHICLE_PARTS,301466,301466
252547,"[{'event_info': 'BOMBA DAGUA SOLAR', 'event_ti...",1012095,Pendente Gota Teat Alumínio Luminária Cores,MLB-WALL_AND_CEILING_LIGHTS,39.90,MLB189195,new,MLB,WALL_AND_CEILING_LIGHTS,252547,252547
148200,"[{'event_info': 1261317, 'event_timestamp': '2...",1703497,Apoio Suporte Cabeça Proteção Bebê Criança Inf...,MLB-HATS_AND_CAPS,54.99,MLB31439,new,MLB,HATS_AND_CAPS,148200,148200
375715,"[{'event_info': 'CUBO MAGICO', 'event_timestam...",271967,Cubo Mágico Profissional 3x3x3 Qiyi Warrior W ...,MLB-PUZZLE_CUBES,44.99,MLB46243,new,MLB,PUZZLE_CUBES,375715,375715


In [21]:
#El siguiente codigo desglosa la columna user_history del df. Se obtiene un nuevo df (final_df)
#que contiene las columnas de df y las columnas event_type, event_timestamp y event_info. 

import time
start_time = time.time()


#total_users=len(x_train)

#x_train['user_id']=np.arange(1,total_users+1)
#

new_df=pd.DataFrame()
new_df2=pd.DataFrame()

print("Data Import: --- %s seconds ---" % (time.time() - start_time))

def clean_json(idx,col):
    parsed = pd.json_normalize(col)
    parsed['user_id']=idx
    return parsed

print('Desglosando user_history...')

for label, content in x_train.user_history.items():
  newcol=clean_json(label,content)
  if label==0:
    new_df=newcol
  else:
    new_df=new_df.append(newcol)
  if (np.mod(label,1000)==0): ## ir cada 1000 parece ser lo más optimo. 
        new_df2 = new_df2.append(new_df)
        del new_df
        new_df=pd.DataFrame()
        
  if (np.mod(label,20000)==0):
        #print('Voy por la fila n°:',label)
        print('Desglosando user_history...')
  del newcol
        

#new_df2.set_index(new_df2.user_id,inplace=True)


#final_df=new_df2.join(x_train,how='left',lsuffix='_desglose')


Data Import: --- 0.000997781753540039 seconds ---
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...
Desglosando user_history...


In [22]:
new_df2.head()

,event_info,event_timestamp,event_type,user_id
0,FIAT STRADA 2015 DUPLA,2019-10-01T21:38:22.372-0400,search,236116
1,433564,2019-10-01T21:38:34.104-0400,view,236116
2,FIAT STRADA 2015 DUPLA,2019-10-01T21:39:44.490-0400,search,236116
3,2078343,2019-10-01T21:40:01.876-0400,view,236116
4,FIAT STRADA 2015 DUPLA,2019-10-01T21:40:37.032-0400,search,236116


In [23]:
new_df2.set_index(new_df2.user_id,inplace=True)
final_df=new_df2.join(x_train,how='left',lsuffix='_desglose')

In [24]:
final_df

,event_info,event_timestamp,event_type,user_id_desglose,user_history,item_id,title,domain_id,price,category_id,condition,country,domain,user_id,user_id2
user_id,,,,,,,,,,,,,,,
0,1786148,2019-10-19T11:25:42.444-0400,view,0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,new,MLB,SMARTWATCHES,0,0
0,1786148,2019-10-19T11:25:57.487-0400,view,0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,new,MLB,SMARTWATCHES,0,0
0,RELOGIO SMARTWATCH,2019-10-19T11:26:07.063-0400,search,0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,new,MLB,SMARTWATCHES,0,0
0,1615991,2019-10-19T11:27:26.879-0400,view,0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,new,MLB,SMARTWATCHES,0,0
0,1615991,2019-10-19T11:28:36.558-0400,view,0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830,Relógio Medidor Inteligente Pulso Freqüência C...,MLB-SMARTWATCHES,90.00,MLB135384,new,MLB,SMARTWATCHES,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413162,1348242,2019-10-04T08:34:56.361-0400,view,413162,"[{'event_info': 'COTONETE FINO', 'event_timest...",1212472,Peeling Ultrassônico E Ionização Portátil Anti...,MLB-SKIN_CARE_SUPPLIES,152.90,MLB200336,new,MLB,SKIN_CARE_SUPPLIES,413162,413162
413162,792798,2019-10-07T10:49:32.217-0400,view,413162,"[{'event_info': 'COTONETE FINO', 'event_timest...",1212472,Peeling Ultrassônico E Ionização Portátil Anti...,MLB-SKIN_CARE_SUPPLIES,152.90,MLB200336,new,MLB,SKIN_CARE_SUPPLIES,413162,413162
413162,258196,2019-10-07T10:52:21.825-0400,view,413162,"[{'event_info': 'COTONETE FINO', 'event_timest...",1212472,Peeling Ultrassônico E Ionização Portátil Anti...,MLB-SKIN_CARE_SUPPLIES,152.90,MLB200336,new,MLB,SKIN_CARE_SUPPLIES,413162,413162


In [25]:
#Eliminamos columnas remanentes que no nos interesan. 
del final_df['user_id_desglose'] 
del final_df['user_history']
del final_df['user_id']
del final_df['domain_id']

El dataset final contiene 6112470 filas. Los datos corresponden a 212943 usuarios (los usuarios de MLB). Por cada usuario aparecen tantas filas como eventos tengan resgitrados durante el tiempo de observacion de su historial. 

In [26]:
final_df.head()

,event_info,event_timestamp,event_type,item_id,title,price,category_id,condition,country,domain,user_id2
user_id,,,,,,,,,,,
0,1786148,2019-10-19T11:25:42.444-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,1786148,2019-10-19T11:25:57.487-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,RELOGIO SMARTWATCH,2019-10-19T11:26:07.063-0400,search,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,1615991,2019-10-19T11:27:26.879-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,1615991,2019-10-19T11:28:36.558-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0


In [27]:
final_df.to_csv('Train_Data_Desglosado.csv') # Guardamos este dataset en caso de que lo necesitemos. 

####Eliminar registros duplicados y analizamos valores nulos. 

In [28]:
#Analizamos si hay filas duplicadas. Para ello, vemos si hay valores repetidos en la columna "time_stamp"
final_df.event_timestamp.duplicated().sum()

9369

No aparecen filas duplicadas. Seguramente algunos usuarios registran el mismo evento realizado en distintos tiempos. Por el momento, no trabajaremos con eso. 

Veamos si aparecen valores nulos. 

In [29]:
final_df.isna().sum()

event_info         0
event_timestamp    0
event_type         0
item_id            0
title              0
price              0
category_id        0
condition          0
country            0
domain             0
user_id2           0
dtype: int64

No se observan valores nulos. 

In [30]:
#Por las dudas, eliminamos los que lleguen a aparecer. 
final_df = final_df.dropna()

In [31]:
final_df

,event_info,event_timestamp,event_type,item_id,title,price,category_id,condition,country,domain,user_id2
user_id,,,,,,,,,,,
0,1786148,2019-10-19T11:25:42.444-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,1786148,2019-10-19T11:25:57.487-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,RELOGIO SMARTWATCH,2019-10-19T11:26:07.063-0400,search,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,1615991,2019-10-19T11:27:26.879-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
0,1615991,2019-10-19T11:28:36.558-0400,view,1748830,Relógio Medidor Inteligente Pulso Freqüência C...,90.00,MLB135384,new,MLB,SMARTWATCHES,0
...,...,...,...,...,...,...,...,...,...,...,...
413162,1348242,2019-10-04T08:34:56.361-0400,view,1212472,Peeling Ultrassônico E Ionização Portátil Anti...,152.90,MLB200336,new,MLB,SKIN_CARE_SUPPLIES,413162
413162,792798,2019-10-07T10:49:32.217-0400,view,1212472,Peeling Ultrassônico E Ionização Portátil Anti...,152.90,MLB200336,new,MLB,SKIN_CARE_SUPPLIES,413162
413162,258196,2019-10-07T10:52:21.825-0400,view,1212472,Peeling Ultrassônico E Ionização Portátil Anti...,152.90,MLB200336,new,MLB,SKIN_CARE_SUPPLIES,413162


Nos queda entonces un dataset de 6112470 filas y 11 columnas. 

In [32]:
#cols_text=final_df.select_dtypes(include=['object', 'category']).columns.to_list()

#### Seleccionamos las columnas con texto (en portugues) e importamos las librerias necesarias.

In [33]:
final_df2=final_df.copy(deep=True)

cols_text=['event_info', 'title', 'domain', 'category_id'] #tokenizamos solo columnas en portugues

import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem.snowball import PortugueseStemmer


##### Definimos las funciones que nos servirán para procesar nuestros textos: 
*Pasamos todo a minuscula: 
*Eliminamos el underscore de la columna "domain"
*Definimimos la función de tokenizacion.
*Removemos las stopwords.
*Aplicamos el PortugueseStemmer
*Armamos una función de funciones ("normalize") que aplica todos estos cambios en un unico paso. 

In [34]:
#https://medium.com/datos-y-ciencia/preprocesamiento-de-datos-de-texto-un-tutorial-en-python-5db5620f1767
#Definimos la funcion que pone todo en minuscula.
final_df2=final_df2.sample(10000)
def minus(cell):
  aux=str(cell).lower() # Lowercase string 
  
  return aux

###########################################################################
# Eliminamos el "guion bajo" de la columna "domain"

#Definimos la funcion que reemplaza _ por un espacio vacio.
def replace(cell):
  aux=str(cell).replace('_',' ') 
  
  return aux

###########################################################################
#Definimos la funcion de tokenizacion. Separa cada celda en una lista de palabras. 

def token(cell):

  aux=nltk.word_tokenize(cell) # Tokenization
  
  return aux

###########################################################################
#Eliminamos palabras del tipo "stopwords" del idioma portugues.
#Son palabras tales como las preposiciones las que se eliminan, palabras que 
#se repiten mucho y sin valor semantico.

def remove_stopwords(cell):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in cell:
        if word not in stopwords.words('portuguese'):
            new_words.append(word)
    return new_words

###########################################################################

# Definimos una funcion que stemmizará cada celda cada celda. 
#Empleamos un stemmizador en portugues.
#Buscamos simplificar las palabras cortando su extremo con la intención de unificar palabras
#con similar significado tales como "reloj", "relojes", "relojeria", por ejemplo, con el fin 
#de lograr comparaciones correctas entre palabras.


def stem_words(cell):
    """Stem words in list of tokenized words"""
    stemmer = PortugueseStemmer()
    stems = []
    for word in cell:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

###########################################################################
#funcion de funciones:

def normalize(words):
    words = minus(words)
    words = replace(words)
    words = token(words)
    words = remove_stopwords(words)
    words = stem_words(words)
    return words



# Modificamos las columnas de texto aplicando las funciones mencionadas.
for col in cols_text:
  print(col)
  final_df2.loc[:,col] = final_df2[col].apply(normalize)  


event_info
title
domain
category_id


In [35]:
final_df2[:20]

,event_info,event_timestamp,event_type,item_id,title,price,category_id,condition,country,domain,user_id2
user_id,,,,,,,,,,,
150862,"[gift, card, fre, fir]",2019-10-02T13:03:34.020-0400,search,1577307,"[bluetooth, p2, receptor, auxil, carr, som, au...",39.45,[mlb60182],new,MLB,"[wireless, fm, transmitters]",150862
160709,[785372],2019-10-08T13:03:44.133-0400,view,1545648,"[transform, tv, smat, min, 4k, android, 9.0, 4...",359.00,[mlb133950],new,MLB,"[streaming, med, devic]",160709
288694,[1440112],2019-09-29T08:36:52.354-0400,view,1713168,"[megalizz, progress, chuveir, formol-, meg, lizz]",139.00,[mlb32130],new,MLB,"[hair, treatments]",288694
130383,[1559476],2019-10-12T19:02:17.353-0400,view,104682,"[microond, philc, 30, litr, pme31, prat, espel...",235.99,[mlb73057],new,MLB,[microwav],130383
64772,[1550604],2019-10-11T19:08:04.520-0400,view,375894,"[selador, manual, 30, cm, -, selador, embalage...",144.00,[mlb49073],new,MLB,"[industrial, and, commercial, bag, sealers]",64772
313256,[2091520],2019-09-30T22:19:09.898-0400,view,1257748,"[lamp, espiã, ip, 360°, hd, panorâm, led, wif,...",81.99,[mlb7073],new,MLB,"[surveillanc, cam]",313256
41104,[1935982],2019-10-18T20:21:12.186-0400,view,1935982,"[cordã, led, pregador, prendedor, varal, clip,...",23.88,[mlb189949],new,MLB,"[christm, lights]",41104
33534,[697409],2019-10-05T11:07:02.392-0400,view,1075533,"[xiaom, mi, 8, mi8, lit, 128gb, 6gb, global, n...",1399.00,[mlb1055],new,MLB,[cellphon],33534
345675,[568392],2019-09-25T17:32:20.420-0400,view,195799,"[kit, 4, top, suplex, nadador, academ, fitness...",34.99,[mlb270229],new,MLB,"[sport, bras]",345675


##### Lemmatizamos.
No encontramos lemmatizador en portugues.

Se busca la raíz de los verbos con el objetivo de eliminar ambiguedades con las distintas conjugaciones verbales. 

In [ ]:
# Definimos una funcion que lemmatizará cada celda

def lemmatize_verbs(cell):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in cell:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [ ]:
#No hemos aplicado la función. Por el momento no la consideramos importante, ya que segun entendemos,
#busca raices de verbos y no tenemos verbos en nuestros textos. 

#for col in cols_text:
 # final_df2.loc[:,col] = final_df2[col].apply(lemmatize_verbs) 

In [ ]:
final_df2.head()